### 모듈 설치
- 아래에 깃 클론 git bash에 치기

In [2]:
%git clone https://github.com/DepthAnything/Depth-Anything-V2

UsageError: Line magic function `%git` not found.


In [2]:
%cd Depth-Anything-V2/metric_depth

d:\DMS\mde-depthAnythingV2\Depth-Anything-V2\metric_depth


C:\Users\smhrd\AppData\Roaming\Python\Python312\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
%pwd

'd:\\DMS\\mde-depthAnythingV2\\Depth-Anything-V2\\metric_depth'

In [ ]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


### 모델 코드 작성
- checkpoint는 깃허브에서 받아와서 metric_depth안에 checkpoints폴더 만들어서 실행

In [4]:
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output


from depth_anything_v2.dpt import DepthAnythingV2

# 모델 설정값
model_configs = {
    'vits': {'encoder': 'vits', 'features': 64, 'out_channels': [48, 96, 192, 384]},
    'vitb': {'encoder': 'vitb', 'features': 128, 'out_channels': [96, 192, 384, 768]},
    'vitl': {'encoder': 'vitl', 'features': 256, 'out_channels': [256, 512, 1024, 1024]}
}

encoder = 'vitl'  # 'vits', 'vitb' 가능
dataset = 'hypersim'  # 'hypersim' (실내), 'vkitti' (실외)
max_depth = 80 if dataset == "vkitti" else 20  # 실내/실외에 따라 max depth 설정

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 모델 로드 (GPU로 이동)
model = DepthAnythingV2(**model_configs[encoder], max_depth = max_depth)
model.load_state_dict(torch.load(f'checkpoints/depth_anything_v2_metric_{dataset}_{encoder}.pth', map_location=device))
model.to(device)
model.eval()

# 이미지 로드 및 전처리 (BGR -> RGB 변환)
raw_img = cv2.imread("images/color35.png")
if raw_img is None:
    raise ValueError("이미지를 불러올 수 없습니다. 경로를 확인하세요.")

raw_img = cv2.cvtColor(raw_img, cv2.COLOR_BGR2RGB)

# 깊이 추론
with torch.no_grad():
    depth = model.infer_image(raw_img)  # 모델 예측

print(depth)

Using device: cpu
[[4.1216617 4.161504  4.2013454 ... 1.967201  2.001041  2.0348766]
 [4.1587353 4.2075634 4.2563906 ... 1.9684844 1.9808365 1.9931872]
 [4.1958084 4.253622  4.3114357 ... 1.9697677 1.960632  1.9514976]
 ...
 [2.1676586 2.1832273 2.1987965 ... 1.803188  1.8123293 1.8214694]
 [2.1743884 2.175673  2.176958  ... 1.798215  1.8125712 1.8269256]
 [2.1811192 2.168118  2.1551168 ... 1.7932414 1.812813  1.8323824]]


In [6]:
def get_depth(event,x,y,flags,param) :
    if event == cv2.EVENT_LBUTTONDOWN:
        if 0 <= x < depth.shape[1] and 0 <= y < depth.shape[0]:
            depth_value = depth[y, x]
            print(f"Clicked at ({x}, {y}) -> Depth: {depth_value:.2f} meters")

cv2.imshow("img",raw_img)
cv2.setMouseCallback("img",get_depth)
cv2.waitKey(0)
cv2.destroyAllWindows()

Clicked at (914, 408) -> Depth: 4.09 meters
Clicked at (329, 620) -> Depth: 5.62 meters
Clicked at (185, 641) -> Depth: 4.44 meters
Clicked at (341, 711) -> Depth: 5.83 meters
Clicked at (388, 806) -> Depth: 4.43 meters
Clicked at (574, 558) -> Depth: 5.23 meters
Clicked at (603, 489) -> Depth: 11.88 meters
Clicked at (625, 414) -> Depth: 12.04 meters
Clicked at (630, 361) -> Depth: 11.74 meters
Clicked at (640, 310) -> Depth: 8.67 meters
Clicked at (637, 293) -> Depth: 8.79 meters
Clicked at (680, 635) -> Depth: 8.51 meters
Clicked at (698, 629) -> Depth: 8.55 meters
Clicked at (932, 395) -> Depth: 4.07 meters
Clicked at (903, 421) -> Depth: 4.07 meters
Clicked at (929, 425) -> Depth: 4.04 meters
Clicked at (924, 421) -> Depth: 4.05 meters
Clicked at (921, 418) -> Depth: 4.07 meters
Clicked at (904, 418) -> Depth: 4.07 meters
Clicked at (901, 423) -> Depth: 4.07 meters
Clicked at (904, 434) -> Depth: 4.06 meters


In [1]:
import tensorflow as tf